### **Imports**

In [1]:
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity

print("✅ Demo tools loaded.")

✅ Demo tools loaded.


### **Model**

In [4]:
# Initialize the open-source embedding model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

print(f"✅ Model '{model_name}' is loaded and ready.")

✅ Model 'sentence-transformers/all-MiniLM-L6-v2' is loaded and ready.


### **Text-to-Vector**
*(Main component of RAG)*

Here is a plain English question being converted to numeric vector.

In [8]:
# Here is a plain English question
question = "How many accidents in Virginia involved alcohol?"

# Let's run it through the model
vector = embeddings.embed_query(question)

print("--- Your Question ---")
print(f"'{question}'")

print("\n--- Becomes a 'Vector' (a list of numbers) ---")
print("\nFirst 5 numbers of the vector:")
print(np.array(vector)[:5])
print(f"\nTotal length of the vector: {len(vector)}")

--- Your Question ---
'How many accidents in Virginia involved alcohol?'

--- Becomes a 'Vector' (a list of numbers) ---

First 5 numbers of the vector:
[0.08255327 0.00739611 0.0154931  0.02287057 0.03947461]

Total length of the vector: 384


In [ ]:
# Now, let's create two other sentences to compare
text_similar = "What is the number of drunk driving crashes in VA?"
text_dissimilar = "What's the weather like in California?"

# Let's embed these as well
vector_similar = embeddings.embed_query(text_similar)
vector_dissimilar = embeddings.embed_query(text_dissimilar)

print("✅ Embedded two more sentences to compare.")

✅ Embedded two more sentences to compare.


In [7]:
# We need to reshape the vectors for the function
v_question = np.array(vector).reshape(1, -1)
v_similar = np.array(vector_similar).reshape(1, -1)
v_dissimilar = np.array(vector_dissimilar).reshape(1, -1)

# Now, let's use 'cosine similarity' to see how "close" they are.
# A score of 1.0 is a perfect match.
sim_similar = cosine_similarity(v_question, v_similar)[0][0]
sim_dissimilar = cosine_similarity(v_question, v_dissimilar)[0][0]

print("--- Similarity Results (1.0 = identical) ---")
print(f"Similarity to 'drunk driving': {sim_similar:.4f}")
print(f"Similarity to 'California weather': {sim_dissimilar:.4f}")

--- Similarity Results (1.0 = identical) ---
Similarity to 'drunk driving': 0.8079
Similarity to 'California weather': 0.2156


### **Retreive Data from Your SQL Database**

In [1]:
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print("✅ RAG tools loaded. We will build this from scratch.")

✅ RAG tools loaded. We will build this from scratch.


### **Load Your FARS Data**

Loading small, fast sample from your CSV to act as our "database."

In [2]:
# Load a sample of your CSV data
df = pd.read_csv("/Users/rafaelviray/Documents/FARS/Datasets & Ingestion Scripts/accident_master.csv", nrows=50)

print("✅ Loaded sample of FARS 'accident_master.csv'.")
print("Here's a preview:")
print(df[['ST_CASE', 'YEAR', 'STATE', 'FATALS']].head())

✅ Loaded sample of FARS 'accident_master.csv'.
Here's a preview:
   ST_CASE  YEAR  STATE  FATALS
0    10001  1975      1       1
1    10002  1975      1       1
2    10003  1975      1       1
3    10004  1975      1       1
4    10005  1975      1       1


### **Building the "R" (Retrieval) Component**

This is the core **"technical content"**. We serialize and index the data.

In [3]:
# 1. Serialize: Convert rows into simple "documents"
print("Serializing CSV rows into text snippets...")
documents = []
for _, row in df.iterrows():
    # Create a simple text snippet for each accident
    content = (
        f"Accident case {row['ST_CASE']} took place in {row['STATENAME']} "
        f"(state code {row['STATE']}) in the year {row['YEAR']}. "
        f"This incident resulted in {row['FATALS']} fatalities."
    )
    doc = Document(page_content=content, metadata={"st_case": row['ST_CASE']})
    documents.append(doc)

# 2. Index: Load the embedding model and build the vector store
print("Loading embedding model...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

print("Building live vector store... (This is the 'R' in RAG)")
vectorstore = Chroma.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

print("✅ Vector store is live.")

Serializing CSV rows into text snippets...
Loading embedding model...


/var/folders/m7/xxvzrlmd0wddkflz97l7nq7r0000gn/T/ipykernel_93503/2808096897.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Building live vector store... (This is the 'R' in RAG)
✅ Vector store is live.


In [4]:
# Initialize our open-source LLM
llm = ChatOllama(model="llama3")

# This is a specific question from our CSV data
question = "How many fatalities were in accident case 47157?"

print(f"--- Asking the 'Dumb' LLM ---")
print(f"QUESTION: {question}\n")

# We invoke the LLM *without* RAG. It will fail.
response = llm.invoke(question)

print("--- RESPONSE (Hallucination) ---")
print(response)

/var/folders/m7/xxvzrlmd0wddkflz97l7nq7r0000gn/T/ipykernel_93503/2675436897.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3")


--- Asking the 'Dumb' LLM ---
QUESTION: How many fatalities were in accident case 47157?

--- RESPONSE (Hallucination) ---
content='I\'m a large language model, I don\'t have access to specific information about accident cases, including the number of fatalities. Additionally, I am not aware of any publicly available information on an accident case with the identifier "47157". If you\'re referring to a specific incident or accident report, please provide more context or details so I can help you better.' additional_kwargs={} response_metadata={'model': 'llama3', 'created_at': '2025-11-05T20:40:10.038412Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 12357539666, 'load_duration': 7508519708, 'prompt_eval_count': 21, 'prompt_eval_duration': 1378666542, 'eval_count': 73, 'eval_duration': 3446750332} id='run--5888b6d6-d308-4f04-ab10-8f6f43221f8e-0'


In [5]:
# 1. Augment: We create a prompt template
# It will be "augmented" with the context we retrieve
template = """
Answer the question based ONLY on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 2. Generate: We build the full chain
def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)

# This chain links all our steps together
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ Full RAG Chain is built.")
print("This chain will now RETRIEVE, AUGMENT, and GENERATE.")

✅ Full RAG Chain is built.
This chain will now RETRIEVE, AUGMENT, and GENERATE.


In [6]:
print(f"--- Asking the 'Smart' RAG Chain ---")
print(f"QUESTION: {question}\n")

# Invoke the full RAG chain
response = rag_chain.invoke(question)

print("--- RESPONSE (Grounded in FARS) ---")
print(response)

--- Asking the 'Smart' RAG Chain ---
QUESTION: How many fatalities were in accident case 47157?

--- RESPONSE (Grounded in FARS) ---
Based on the provided context, there is no information about accident case 47157. Therefore, it is not possible to determine the number of fatalities for this accident case. The only available information pertains to cases 10014, 10015, 10026, and 10033.
